# PGM Laboratory: Classification with Tree Structured Models

Author: Aaron Helmers

This notebook requires:
* numpy  
* pandas  
* scipy  
* pymc3  
* patsy  

**warning**: pymc3 can be difficult to install. I was able to get it installed with [these instructions](http://stackoverflow.com/questions/42728877/using-pymc3-on-windows-10-theano-cannot-import-name-floatx?noredirect=1#comment72639271_42728877) along with [these instructions](http://deeplearning.net/software/theano/install_windows.html) taking the updated theanos from the former instruction set over the later.

## Problem statement
This laboratory comes from Daphne Kollor's Stanford Course on Probabalistic Graphical Models (PGM).

The goal is to be able to classify a body based on its pose. To do this, we have access to data for 10 different parts of the body: torso (0), head(1), left arm (2), left forearm (3), right arm (4), right forearm (5), left thigh (6), left leg (7), right thigh (8), and right leg (9). For each body part we also have access to 3 parameters: Y position, X position, and rotation (alpha).

The laboratory juxtaposes 2 methods of using this data to classify the body: A naive bayes model, and a conditional linear gaussian model (CLG).

## Load Data
The data we are loading was originally meant for matlab. To load the data, we will use scipy's io capabilities. The result of the load is a dictionary, where each key->value represents matrix_name->matrix. The matricies we are loading are:
* G1: The tree structure of a regular bayesian network, where each body part has only the body classification (class priors) as it's parent node.
* G2: The tree structure representing bodily connections for the CLG method of classification. This is to say that each body part, besides the torso, has another body part as a parent to learn from. all body parts also have the body classification as a parent. The structure of the data is [body_part][parent_type, parent].
* trainData / testData: The actual input data for our use. The structure of the data is [sample][body_part][parameter].

scipy.io loads the data strangly (there are extra, unneeded dimensions prepended), so we also extract it to the expected format listed above.

In [3]:
import scipy.io as sio

data = sio.loadmat('data/PA8Data.mat')
data['trainData'] = data['trainData'][0][0]
data['testData'] = data['testData'][0][0]
print (list(data))

['__header__', '__version__', '__globals__', 'G1', 'G2', 'trainData', 'testData']


Next, we want to convert from 1d numpy arrays to pandas dataframes. Since DataFrames must be 2D, we will create a dictionary from body_part->(sample, parameters)

In [4]:
import pandas as pd
import numpy as np
# reshape input data from [s,b,p] to {b, df(s,p)}
def reshape_data(samples):
    # convert from [s][b][p] to [s,b,p]
    retData = np.array(list(map(
        lambda sample: np.array(np.array(sample)),
        samples
    )))
    # convert from [s,b,p] to [b,s,p]
    retData = np.swapaxes(retData, 0, 1)
    # convert from [b,s,p] to {b: df(s,p)}
    retData = dict(map(
        lambda part_index: (part_index, pd.DataFrame(retData[part_index,:,:])),
        range(0, retData.shape[0])
    ))
    # give each DF proper column names
    for part in retData: 
        retData[part].columns = ['y', 'x', 'alpha']
    return retData

trainData = list(map(
    lambda label: reshape_data(
        data['trainData']['data'][np.where(data['trainData']['labels'][:,0] == label)]
    ), [0, 1]
))
testData = list(map(
    lambda label: reshape_data(
        data['testData']['data'][np.where(data['testData']['labels'][:,0] == label)]
    ), [0, 1]
    
))

print("Training Data (class=human(0), part=head(1)):")
print(trainData[0][1][:5])
print("\nTesting Data (class=human(0), part=head(1)):")
print(testData[0][1][:5])

Training Data (class=human(0), part=head(1)):
           y          x     alpha
0  22.977837   8.273512 -6.145974
1   8.363819  -1.982919 -0.115858
2  21.482629  38.584170 -6.267717
3 -19.544711 -24.646000 -5.760630
4  18.078155   8.942255 -5.676765

Testing Data (class=human(0), part=head(1)):
           y          x     alpha
0   0.688736  -9.333963 -0.374123
1  27.601101 -19.407871 -0.465951
2  19.320989  -6.411027 -0.481047
3 -23.182955  -1.804526 -6.082243
4 -19.615777  18.688371 -5.888298


## Develop Model
For the purpose of this laboratory, only the CLG method will be explored (G2). Originally G1 was included to juxtapose the naive bayes method with the CLG method.

The CLG method can be seen as determining the probability of a classification for the body by finding the probability of all body parts having that classification. Subsequently, each body part determine's it's probability of classification based on learned linear gaussian's whose mean is a linear combination the the classes prior probability the parent body part's parameters.

While doing research to complete this laboratory, 3 ways were found to fulfil the requirements:

1) The hardest is to write the models manually, which includes the writing of a method to solve the linear systems for each body_part-parameter-classification combination, or 60 linear systems in total. Then follow this up by writing a method to compute the log-likelihood for a testing sample to classify.  

2) pgmpy could be used to make life easier by creating a LinearGaussianCPD model for each body_part-parameter-classification combination. The benifit of this is that it makes the calculation of the log-likelihood much simpler by providing the probability of classification for each model. These models would then just have to be combined to compute the probability of classification. To create these models, however, we would still need to compute the parameters for each gaussian, which is a significant amount of work.  

3) pymc3 was then discovered which was capable of providing Generalized Linear Models (GLMs), which is precisely what we are looking to create. These take the capability of pgmpy above, but abstract away the work required to learn the gaussian parameters.      

Since pymc3 abstracts a lot of the complicated work away, it was used to solve our problems.

### 1) Reformulate Data Into Linear Gaussian Form
In order to utilize Pymc3, we need to reformulate each of our samples into (y<sub>parent</sub>, x<sub>parent</sub>, alpha<sub>parent</sub>, answer) form. This will allow us to have Pymc3 create models for each body part's parameter. The offset parameter is automatically calculated for us by Pymc3.

Note that this format is only considered for all but the root body part (torso), as the root body part has no parent to parameterize off of. The torso will instead be modeled by a gaussian with the expectation and variance of the parameter being modeled.

In [5]:
def as_paramed(data_in, class_prior):
    return dict(map(
        lambda part_index: (part_index, pd.DataFrame({
            'Intercept': class_prior,
            'p_x': data_in[data['G2'][1,1]]['x'],
            'p_y': data_in[data['G2'][1,1]]['y'],
            'p_alpha':data_in[data['G2'][1,1]]['alpha'],
            'x': data_in[part_index]['x'],
            'y': data_in[part_index]['y'],
            'alpha': data_in[part_index]['alpha']
        })),
        range(1, len(data_in))
    ))

trainDataParamed = list(map(
    lambda label: as_paramed(
        trainData[label],
        trainData[label][0].shape[0] / data['trainData']['data'].shape[0]
    ), [0, 1]
))
testDataParamed = list(map(
    lambda label: as_paramed(
        testData[label],
        testData[label][0].shape[0] / data['trainData']['data'].shape[0]
    ), [0, 1]
))

print('Parameterized Training Data (class=human(0), part=head(1)):')
print(trainDataParamed[0][1][:5])
print('\nParameterized Testing Data (class=alien(1), part=right_leg(9)):')
print(trainDataParamed[1][9][:5])

Parameterized Training Data (class=human(0), part=head(1)):
   Intercept     alpha   p_alpha        p_x        p_y          x          y
0   0.488889 -6.145974 -6.145974   8.273512  22.977837   8.273512  22.977837
1   0.488889 -0.115858 -0.115858  -1.982919   8.363819  -1.982919   8.363819
2   0.488889 -6.267717 -6.267717  38.584170  21.482629  38.584170  21.482629
3   0.488889 -5.760630 -5.760630 -24.646000 -19.544711 -24.646000 -19.544711
4   0.488889 -5.676765 -5.676765   8.942255  18.078155   8.942255  18.078155

Parameterized Testing Data (class=alien(1), part=right_leg(9)):
   Intercept     alpha   p_alpha        p_x        p_y          x           y
0   0.511111 -2.784688 -5.659289  -8.290587 -30.099826 -27.125302   80.679541
1   0.511111 -3.467803 -0.168724 -26.986419 -35.347895  -2.146990   71.836016
2   0.511111 -2.818784 -6.181410   9.526765  22.749323 -30.957147  127.921171
3   0.511111 -3.585914 -6.281134   2.292619 -21.035900  42.804573   79.490427
4   0.511111 -3.184570 

### 2) Build Pymc3 Generalized Linear Models
We are going to start by building GLMs for each body part and classification, resulting in 20 GLMs in total. A normal probability distribution is assumed.

In [6]:
# with glm[0][1]['x']:

In [7]:
from theano import shared

In [8]:
import pymc3 as pm
from theano import shared


def generate_param_glm(param, samples):
    with pm.Model() as model:
         pm.glm.glm(param + ' ~ p_y + p_x + p_alpha', samples)
    return model

def generate_body_glms(samples):
    return dict(map(
        lambda param: (param, generate_param_glm(param, samples)),
        ['y', 'x', 'alpha']
    ))

# Generate distributions as [{part: {param: model}}] where index is a classification (0, 1)
glms = list(map(
    lambda label: dict(map(
        lambda part: (part, generate_body_glms(trainDataParamed[label][part])),
        range(1, len(trainData[label]))
    )),
    [0, 1]
))

# Display
print (' GLMs for (label=human, part=head):')
print (glms[0][1])

 GLMs for (label=human, part=head):
{'y': <pymc3.model.Model object at 0x0000024ACF3CCF60>, 'x': <pymc3.model.Model object at 0x0000024ACF3CCF98>, 'alpha': <pymc3.model.Model object at 0x0000024AD0123588>}


### 3) Define Methodology for computing Log Likelihood by Classification 

In [10]:
def get_part_probability(label, part, samples):
    param_probs = list(map(
        lambda param: glms[label][part][param].logp(sample),
        list(samples)
    ))
    return param_probs

def get_trace(label, part, param, in_data):
    with glms[label][part][param]:
        trace = pm.sample(in_data.shape[0], start=testData[0][1])
    return trace

with glms[0][1]['x']:
    trace = pm.sample(200)
trace

Auto-assigning NUTS sampler...
Initializing NUTS using advi...

Average ELBO = 671.7: 100%|██████████████████████████████████| 200000/200000 [00:21<00:00, 9481.74it/s]Finished [100%]: Average ELBO = 671.05

100%|████████████████████████████████████████████████████████████████| 200/200 [00:02<00:00, 70.59it/s]

<MultiTrace: 1 chains, 200 iterations, 6 variables>

In [12]:
from pymc3 import traceplot
import matplotlib.pyplot as plt

traceplot(trace);